## 格式化时间与icao24

In [9]:
import pandas as pd
import numpy as np
import os

data = pd.read_csv('filter_data.csv')
del data['date']
unique_icao24 = data['icao24'].unique()
len(data)
# 导出修改过reg之后的数据
# for i in range(len(data)):
#     data['reg'] = str(i)
# data['time'].min()
# data.replace()
# 按某一列的值进行排序
data.sort_values('time',inplace=True)
time_min = 1587340810
# 对单列数据操作
data['time'] = data['time'].map(lambda x:int(x) - time_min)
data['icao24'] = data['icao24'].map(lambda x: list(unique_icao24).index(x))
# 格式化数据
lat_min =data['lat'].min()
lat_max = data['lat'].max()
lon_min =data['lon'].min()
lon_max = data['lon'].max()
geoaltitude_min =data['geoaltitude'].min()
geoaltitude_max = data['geoaltitude'].max()
data['lat'] = data['lat'].apply(lambda x: (x - lat_min)/(lat_max - lat_min))
data['lon'] = data['lon'].apply(lambda x: (x - lon_min)/(lon_max - lon_min))
data['geoaltitude'] = data['geoaltitude'].apply(lambda x: (x - geoaltitude_min)/(geoaltitude_max - geoaltitude_min))

# 过滤掉航迹长度小于8的行
data = data.groupby('icao24').filter(lambda x: len(x) >= 8)

In [10]:
## 将训练集、验证集、测试集按照6:2:2划分
# 随机抽样
# train, validate, test = np.split(data.sample(frac=1), [int(.7*len(data)), int(.9*len(data))])

train_row = int(.6*len(data))
val_row = int(.8*len(data))

train = data.iloc[:train_row]
val = data.iloc[train_row:val_row]
test = data.iloc[val_row:]


dataset_path = 'states_2020-04-20'
train_path = os.path.join(dataset_path,"train")
val_path = os.path.join(dataset_path,"val")
test_path = os.path.join(dataset_path,"test")

if not os.path.exists(train_path):
    os.makedirs(train_path)

if not os.path.exists(val_path):
    os.makedirs(val_path)

if not os.path.exists(test_path):
    os.makedirs(test_path)

train_file = os.path.join(train_path, 'train.txt')
val_file = os.path.join(val_path, 'val.txt')
test_file = os.path.join(test_path, 'test.txt')
# 保存文件
train.to_csv(train_file, index=False, sep='\t')
val.to_csv(val_file, index=False, sep='\t')
test.to_csv(test_file, index=False, sep='\t')